In [1]:
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader, Dataset
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
from math import ceil

In [8]:
B, T, D = 14, 20770, 64
cond_num = 1
update_ix = np.load('C:\\Users\\kdmen\\Desktop\\Research\\Data\\update_ix.npy')
starting_update = 10
final_update = 17
batch_size = 14
num_tensor_dims = 3
num_users = 14

class CustomTimeSeriesDataset(Dataset):
    def __init__(self, data, labels, num_dims):
        self.data = data
        self.labels = labels
        self.num_dims = num_dims
        assert ((self.num_dims == 2) or (self.num_dims == 3))

    def __len__(self):
        if self.num_dims==2:
            return self.data.shape[1] # Return the number of observations (sequence length)
        elif self.num_dims==3:
            return self.data.shape[0] # Return the number of sequences

    def __getitem__(self, idx):
        if self.num_dims==2:
            sample_data = torch.Tensor(self.data[:,idx])
            sample_labels = torch.Tensor(self.labels[:,idx])
            return sample_data, sample_labels
        elif self.num_dims==3:
            sample_data = torch.Tensor(self.data[idx])
            sample_labels = torch.Tensor(self.labels[idx])
            return sample_data, sample_labels

# Load Data
print("Loading Data")
input_data = None
target_data = None
data_path = r"C:\\Users\\kdmen\\Desktop\\Research\\Data\\Client_Specific_Files"
for i in range(num_users):
    datafile = "UserID" + str(i) + "_TrainData_8by20770by64.npy"
    full_data = np.load(data_path+"\\"+datafile)
    cond_data = full_data[cond_num-1, update_ix[starting_update]:update_ix[final_update], :]
    data = np.transpose(cond_data)
    if input_data is None:
        input_data = data
    else:
        input_data = np.vstack((input_data, data))

    labelfile = "UserID" + str(i) + "_Labels_8by20770by2.npy"
    full_data = np.load(data_path+"\\"+labelfile)
    cond_data = full_data[cond_num-1, update_ix[starting_update]:update_ix[final_update], :]
    data = np.transpose(cond_data)
    if target_data is None:
        target_data = data
    else:
        target_data = np.vstack((target_data, data))

#################

test_split_idx = ceil(input_data.shape[1]*.8)
test_data = torch.tensor(input_data[:, test_split_idx:], dtype=torch.float)
test_labels = torch.tensor(target_data[:, test_split_idx:], dtype=torch.float)
train_data = torch.tensor(input_data[:, :test_split_idx], dtype=torch.float)
train_labels = torch.tensor(target_data[:, :test_split_idx], dtype=torch.float)
print("Data loaded!")

if num_tensor_dims == 3:
    # Calculate the size for the reshaped tensor
    test_data_new_size = (num_users, -1, test_data.shape[1])
    test_labels_new_size = (num_users, -1, test_labels.shape[1])
    train_data_new_size = (num_users, -1, train_data.shape[1])
    train_labels_new_size = (num_users, -1, train_labels.shape[1])
    # Reshape the tensor
    test_data = test_data.view(*test_data_new_size)
    test_labels = test_labels.view(*test_labels_new_size)
    train_data = train_data.view(*train_data_new_size)
    train_labels = train_labels.view(*train_labels_new_size)

#################

# Convert data to DataLoader
print("Create custom datasets")
train_dataset = CustomTimeSeriesDataset(train_data, train_labels, 3)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
test_dataset = CustomTimeSeriesDataset(test_data, test_labels, 3)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
inference_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
print("Datasets and dataloaders created!")

Loading Data
Data loaded!
Create custom datasets
Datasets and dataloaders created!


In [9]:
print(train_data.shape)
print(train_labels.shape)

torch.Size([14, 64, 6732])
torch.Size([14, 2, 6732])


In [10]:
train_i = 0
for i, (batch_data, batch_labels) in enumerate(train_loader):
    if i==0:
        print(f"Batch {i + 1} - Data Shape: {batch_data.shape}, Labels Shape: {batch_labels.shape}")
    train_i += 1
print(train_i)

print()

test_i = 0
for i, (batch_data, batch_labels) in enumerate(test_loader):
    if i==0:
        print(f"Batch {i + 1} - Data Shape: {batch_data.shape}, Labels Shape: {batch_labels.shape}")
    test_i += 1
print(test_i)

print()

inf_i = 0
for i, (batch_data, batch_labels) in enumerate(inference_loader):
    if i==0:
        print(f"Batch {i + 1} - Data Shape: {batch_data.shape}, Labels Shape: {batch_labels.shape}")
    inf_i += 1
print(inf_i)

Batch 1 - Data Shape: torch.Size([14, 64, 6732]), Labels Shape: torch.Size([14, 2, 6732])
1

Batch 1 - Data Shape: torch.Size([14, 64, 1682]), Labels Shape: torch.Size([14, 2, 1682])
1

Batch 1 - Data Shape: torch.Size([1, 64, 1682]), Labels Shape: torch.Size([1, 2, 1682])
14


In [27]:
# Initialize the model
input_size = D  # Number of features in the input data
output_size = 2  # Number of dimensions in the output labels
learning_rate = 0.005

## Linear Regression

In [28]:
from utils.custom_loss_class import CPHSLoss

lambdaF=0
lambdaD=1e-3
lambdaE=1e-4
criterion = CPHSLoss(lambdaF=lambdaF, lambdaD=lambdaD, lambdaE=lambdaE)

In [39]:
def normalize_tensor(input_tensor):
    '''Normalizes a tensor of any dimensions. Goal is to have inputs on the range 0-1, NOT a norm of 1.'''
    
    # Compute min and max values across all dimensions
    min_values = torch.min(input_tensor)
    max_values = torch.max(input_tensor)

    return (input_tensor - min_values) / (max_values - min_values)

In [ ]:
#def simulate_data_streaming_xy(self, x, y, test_data=False):
s_temp = x
p_reference = torch.transpose(y, 0, 1)

s_normed = normalize_tensor(s_temp)
p_reference = normalize_tensor(p_reference)

self.F = s[:,:-1]
v_actual =  torch.matmul(self.model.weight, s)
# Numerical integration of v_actual to get p_actual
p_actual = torch.cumsum(v_actual, dim=1)*self.dt
# I don't think I actually use V later on
self.V = (p_reference - p_actual)*self.dt
self.y_ref = p_reference[:, :-1]  # To match the input

In [30]:
# Define a simple linear regression model
class LinearRegressionModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(LinearRegressionModel, self).__init__()
        self.linear = nn.Linear(input_size, output_size)

        # Initialize the weights using a specific initialization method
        nn.init.xavier_normal_(self.linear.weight)

    def forward(self, x):
        return self.linear(x)

model = LinearRegressionModel(input_size, output_size)

# Define loss function and optimizer
#criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# Training loop
num_epochs = 10
train_loss_log = []
for epoch in range(num_epochs):
    epoch_loss = 0.0
    num_batches = len(train_loader)
    
    for batch_data, batch_labels in train_loader:
        # Flatten the input data
        batch_data = batch_data.view(-1, input_size)

        # Forward pass
        outputs = model(batch_data)

        # LOSS
        # L2 regularization term
        l2_loss = 0
        for name, param in model.named_parameters():
            if 'weight' in name:
                l2_loss += torch.norm(param, p=2)
        t1 = criterion(outputs, batch_labels.view(-1, output_size))
        t2 = lambdaD*(l2_loss**2)
        print(f"t1: {t1}")
        print(f"t2: {t2}")
        #t3 = lambdaF*(torch.linalg.matrix_norm((F))**2)
        t3 = 0 
        loss = t1 + t2 + t3
        epoch_loss += loss.item()
        if np.isnan(epoch_loss) or np.isinf(epoch_loss):
            #print(f"t1: {t1}")
            #print(f"t2: {t2}")
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
            raise ValueError(f"EPOCH_LOSS IS {epoch_loss}")

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Log average loss acros all batches for the epoch
    average_epoch_loss = epoch_loss / num_batches
    train_loss_log.append(epoch_loss)

    #if epoch%10==0:
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Plot the training loss over epochs
plt.plot(range(1, num_epochs+1), train_loss_log, label='Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss Over Epochs')
plt.legend()
plt.show()

t1: 17071574.0
t2: 0.0035664208699017763
Epoch [1/10], Loss: 17071574.0000
t1: 1.5797770973257662e+19
t2: 164185152.0
Epoch [2/10], Loss: 15797770973257662464.0000
t1: 1.462421364381834e+31
t2: 1.519892218509179e+20
Epoch [3/10], Loss: 14624213643818340766111200444416.0000
t1: inf
t2: 1.4070001911769784e+32
Epoch [4/10], Loss: inf


ValueError: EPOCH_LOSS IS inf

In [19]:
for param in model.parameters():
    print(param)

Parameter containing:
tensor([[-9.2567e+16, -9.2485e+16, -9.2420e+16, -9.2475e+16, -9.2543e+16,
         -9.2378e+16, -9.2185e+16, -9.2169e+16, -9.2342e+16, -9.2212e+16,
         -9.2146e+16, -9.2154e+16, -9.2317e+16, -9.2170e+16, -9.1953e+16,
         -9.2124e+16, -9.2307e+16, -9.2237e+16, -9.2069e+16, -9.2113e+16,
         -9.2331e+16, -9.2218e+16, -9.2080e+16, -9.2279e+16, -9.2498e+16,
         -9.2406e+16, -9.2327e+16, -9.2354e+16, -9.2485e+16, -9.2339e+16,
         -9.2186e+16, -9.2288e+16, -9.2506e+16, -9.2349e+16, -9.2212e+16,
         -9.2209e+16, -9.2205e+16, -9.2072e+16, -9.2013e+16, -9.2104e+16,
         -9.2198e+16, -9.1965e+16, -9.1908e+16, -9.2107e+16, -9.2192e+16,
         -9.2101e+16, -9.1954e+16, -9.2125e+16, -9.2321e+16, -9.2205e+16,
         -9.2172e+16, -9.2085e+16, -9.2131e+16, -9.1977e+16, -9.1976e+16,
         -9.2197e+16, -9.2401e+16, -9.2230e+16, -9.2114e+16, -9.2253e+16,
         -9.2428e+16, -9.2281e+16, -9.2206e+16, -9.2217e+16],
        [ 7.5466e+16,  7.540

In [15]:
outputs

tensor([[-1.9057e+20,  1.5536e+20],
        [-1.8936e+20,  1.5438e+20],
        [-1.8660e+20,  1.5213e+20],
        ...,
        [-1.5690e+20,  1.2792e+20],
        [-1.2507e+20,  1.0197e+20],
        [-1.7177e+20,  1.4004e+20]], grad_fn=<AddmmBackward0>)

In [16]:
batch_labels

tensor([[[  5.6089,   5.6089,   2.9489,  ...,  -3.9273,  -3.9273,  -3.9273],
         [  5.3464,   5.3464,   5.3166,  ...,   7.0196,   7.0196,   7.0196]],

        [[-15.4448, -15.4448, -10.8032,  ...,  21.1627,  21.7610,  21.7610],
         [-21.7431, -21.7431, -19.9325,  ...,  14.5263,  14.7706,  14.7706]],

        [[  7.7023,   7.7023,   5.9858,  ...,  -3.1351,  -3.9392,  -3.9392],
         [ -7.7380,  -7.7380,  -9.0258,  ...,  10.2470,  11.1555,  11.1555]],

        ...,

        [[ 34.4756,  34.6800,  35.0770,  ..., -18.0856, -18.0856, -18.5382],
         [ 14.9993,  15.8593,  18.6275,  ...,  -4.2904,  -4.2904,  -4.4546]],

        [[-33.0132, -33.4118, -35.9532,  ...,   6.0055,   6.1073,   6.3316],
         [ 14.1637,  14.1922,  14.1270,  ...,  -0.2625,  -0.2113,  -0.0874]],

        [[-18.4834, -18.6501, -19.7785,  ...,  12.1055,  11.4284,  11.0979],
         [-17.5188, -17.8982, -20.2428,  ...,  21.1333,  20.7342,  20.5140]]])

In [ ]:
# Testing loop
model.eval()
with torch.no_grad():
    total_loss = 0
    for batch_data, batch_labels in test_loader:
        print(batch_data.shape)
        batch_data = batch_data.view(-1, input_size)
        print(batch_data.shape)
        outputs = model(batch_data)
        print(outputs.shape)
        total_loss += criterion(outputs, batch_labels.view(-1, output_size)).item()
        print()

    average_loss = total_loss / len(test_loader)
    print(f'Test Loss: {average_loss:.4f}')

In [ ]:
plt.plot(outputs[:,0])
plt.plot(batch_labels.view(-1, output_size)[:,0])

Single at a time

In [ ]:
# Testing loop
model.eval()
outputs_log = []
labels_log = []
with torch.no_grad():
    total_loss = 0
    for batch_data, batch_labels in test_loader:
        print(f"batch_data.shape: {batch_data.shape}")
        for user_num in range(batch_data.shape[0]):
            user1_seq_data = batch_data[user_num,:,:]
            user1_seq_labels = batch_labels[user_num,:,:]
            print(f"user1_seq_data.shape: {user1_seq_data.shape}")
            iter_i = 0
            for (one_seq, one_seq_label) in zip(user1_seq_data, user1_seq_labels):
                iter_i += 1
                outputs = model(one_seq)
                outputs_log.append(outputs[0].item())
                labels_log.append(one_seq_label[0].item())
                if iter_i%1000==0:
                    #print(f"outputs.shape: {outputs.shape}")
                    print(f"{outputs}, {one_seq_label}")
                total_loss += criterion(outputs, one_seq_label).item()
            print()

    average_loss = total_loss / len(test_loader)
    print(f'Test Loss: {average_loss:.4f}')

In [ ]:
plt.plot(outputs_log)
plt.plot(labels_log)

> Above, I show that you get the same outputs when you use the same model, whether or not you compute all the data at once (eg the default) or doing it one by one

## RNN

In [ ]:
#input_size = D  # Number of features in the input data
#output_size = 2  # Number of dimensions in the output labels
hidden_size = 64

In [ ]:
# Define a simple RNN model
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNNModel, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # RNN input: (batch_size, seq_len, input_size)
        # Output: (batch_size, seq_len, hidden_size)
        rnn_out, _ = self.rnn(x)

        # Fully connected layer
        output = self.fc(rnn_out)
        return output

# Initialize the RNN model
rnn_model = RNNModel(D, hidden_size, 2)  # Change output_size to 2

# Define loss function and optimizer for the RNN model
#rnn_criterion = nn.MSELoss()
rnn_criterion = CPHSLoss(lambdaF=lambdaF, lambdaD=lambdaD, lambdaE=lambdaE)
rnn_optimizer = optim.SGD(rnn_model.parameters(), lr=0.01)

# Training loop for the RNN model
rnn_train_losses = []

num_epochs = 100
for epoch in range(num_epochs):
    rnn_epoch_losses = []
    batch_counter = 0
    for batch_data, batch_labels in train_loader:
        #print(f"Pre-reshape batch_data size: {batch_data.shape}")
        # Reshape the input data to (batch_size, seq_len, input_size)
        #batch_data = batch_data.permute(0, 2, 1)
        #print(f"Post-reshape batch_data size: {batch_data.shape}")

        # Make sure the input size matches the RNN input size
        assert batch_data.size(-1) == D, f"Expected input size {D}, got {batch_data.size(-1)}"

        # Forward pass
        rnn_outputs = rnn_model(batch_data)

        if epoch==0:
            print(f"Batch {batch_counter}. batch_data size: {batch_data.shape}. batch_labels size: {batch_labels.shape}. rnn_outputs size: {rnn_outputs.shape}.")
        batch_counter += 1
        
        rnn_loss = rnn_criterion(rnn_outputs, batch_labels)

        # Backward and optimize
        rnn_optimizer.zero_grad()
        rnn_loss.backward()
        rnn_optimizer.step()

        rnn_epoch_losses.append(rnn_loss.item())

    average_rnn_epoch_loss = sum(rnn_epoch_losses) / len(rnn_epoch_losses)
    rnn_train_losses.append(average_rnn_epoch_loss)
    if epoch%10==0:
        print(f'RNN Epoch [{epoch+1}/{num_epochs}], Loss: {average_rnn_epoch_loss:.4f}')

# Plot the training loss for the RNN model over epochs
plt.plot(range(1, num_epochs+1), rnn_train_losses, label='RNN Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('RNN Training Loss Over Epochs')
plt.legend()
plt.show()


In [ ]:
print("TRAIN LOADER")
model.eval()
with torch.no_grad():
    dim0 = 0
    dim1 = 0
    dim2 = 0
    batch_counter = 0
    for batch_data, batch_labels in train_loader:
        print(batch_counter)
        batch_counter += 1
        print(f"Pre-reshape batch_data size: {batch_data.shape}")
        print(f"Pre-reshape batch_labels size: {batch_labels.shape}")
        dim0 += batch_data.shape[0]
        dim1 += batch_data.shape[1]
        dim2 += batch_data.shape[2]
        
        print()
    print(f"Summed dims: ({dim0}, {dim1}, {dim2})")

In [ ]:
print("TEST LOADER")
model.eval()
with torch.no_grad():
    dim0 = 0
    dim1 = 0
    dim2 = 0
    for batch_data, batch_labels in test_loader:
        print(f"Pre-reshape batch_data size: {batch_data.shape}")
        print(f"Pre-reshape batch_labels size: {batch_labels.shape}")
        dim0 += batch_data.shape[0]
        dim1 += batch_data.shape[1]
        dim2 += batch_data.shape[2]
        
        print()
    print(f"Summed dims: ({dim0}, {dim1}, {dim2})")

In [ ]:
# Testing loop
model.eval()
with torch.no_grad():
    total_loss = 0
    for batch_data, batch_labels in test_loader:
        print(f"batch_data size: {batch_data.shape}")
        print(f"batch_labels size: {batch_labels.shape}")
        outputs = rnn_model(batch_data)
        #reshaped_batch_labels = batch_labels.view(-1, output_size)
        print(f"outputs size: {outputs.shape}")
        #print(f"reshaped_batch_labels size: {reshaped_batch_labels.shape}")
        total_loss += criterion(outputs, batch_labels).item()

    average_loss = total_loss / len(test_loader)
    print(f'Test Loss: {average_loss:.4f}')

## LSTM

In [ ]:
# Define a simple LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # LSTM input: (batch_size, seq_len, input_size)
        # Output: (batch_size, seq_len, hidden_size)
        lstm_out, _ = self.lstm(x)

        # Take the last time step's output
        last_output = lstm_out#[:, , :]

        # Fully connected layer
        output = self.fc(last_output)
        return output

# Initialize the LSTM model
hidden_size = 64
lstm_model = LSTMModel(D, hidden_size, output_size)  # Change input_size to D

# Define loss function and optimizer for the LSTM model
#lstm_criterion = nn.MSELoss()
lstm_criterion = CPHSLoss(lambdaF=lambdaF, lambdaD=lambdaD, lambdaE=lambdaE)
lstm_optimizer = optim.SGD(lstm_model.parameters(), lr=learning_rate)

# Training loop for the LSTM model
lstm_train_losses = []

num_epochs = 100
for epoch in range(num_epochs):
    lstm_epoch_losses = []
    for batch_data, batch_labels in train_loader:
        # Reshape the input data to (batch_size, seq_len, input_size)
        #batch_data = batch_data.view(batch_data.size(0), -1, D)

        # Make sure the input size matches the LSTM input size
        assert batch_data.size(-1) == D, f"Expected input size {D}, got {batch_data.size(-1)}"

        # Forward pass
        lstm_outputs = lstm_model(batch_data)
        lstm_loss = lstm_criterion(lstm_outputs, batch_labels)

        # Backward and optimize
        lstm_optimizer.zero_grad()
        lstm_loss.backward()
        lstm_optimizer.step()

        lstm_epoch_losses.append(lstm_loss.item())

    average_lstm_epoch_loss = sum(lstm_epoch_losses) / len(lstm_epoch_losses)
    lstm_train_losses.append(average_lstm_epoch_loss)
    if epoch%10==0:
        (f'LSTM Epoch [{epoch+1}/{num_epochs}], Loss: {average_lstm_epoch_loss:.4f}')

# Plot the training loss for the LSTM model over epochs
plt.plot(range(1, num_epochs+1), lstm_train_losses, label='LSTM Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('LSTM Training Loss Over Epochs')
plt.legend()
plt.show()

In [ ]:
# Testing loop
model.eval()
with torch.no_grad():
    total_loss = 0
    for batch_data, batch_labels in test_loader:
        batch_data = batch_data.view(-1, input_size)
        outputs = model(batch_data)
        total_loss += criterion(outputs, batch_labels.view(-1, output_size)).item()

    average_loss = total_loss / len(test_loader)
    print(f'Test Loss: {average_loss:.4f}')